Importamos las librerías que vamos a necesitar

In [ ]:
import numpy as np
import ctypes
import time
import statistics
import matplotlib.pyplot as plt

In [ ]:
value = np.random.randint(10)

Creamos nuestra función de varianza en C

In [ ]:
%%file cLimite.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

double cLimite(int N){
    double f=1;
    for(int i = 0; i<N; i++){
        double tmp = 1 + 1/(double) N;
        f *= tmp;
    }
    return f;
}

Linkeamos el ejecutable de C como una librería compartida para usar en python

In [ ]:
! gcc -c -Wall -Werror -fpic cLimite.c
! gcc -shared cLimite.o -o _cLimite.so

Definimos la función con ctypes en python

In [ ]:
def ctypes_cLimite():
    # ruta de la shared library
    lib = ctypes.CDLL('./_cLimite.so')
    
    # tipo de los argumentos
    lib.cLimite.argtypes = [
        ctypes.c_int
    ]

    # definimos el tipo de salida de la función
    lib.cLimite.restype = ctypes.c_double

    # se devuelve la función configurada
    return lib.cLimite

In [ ]:
cLimite = ctypes_cLimite()

Probamos nuestra función

In [ ]:
print("Valor:",end = " ")
print(value)
var = cLimite(value)
print("Limite:",end = " ")
print(var)

Implementamos la misma función en lenguaje ensamblador

In [ ]:
%%file asmLimite.asm
global asmLimite  
section .text
asmLimite:
    xorpd xmm1,xmm1
    xorpd xmm2,xmm2
    xorpd xmm3,xmm3
    mov r8, 1
    cvtsi2sd xmm3,r8
    cvtsi2sd xmm0, rdi
base:
    movsd xmm1, xmm3
    divsd xmm1, xmm0
    addsd xmm1, xmm3
    mov rcx, rdi
    movsd xmm2,xmm3
power:
    mulsd xmm2,xmm1
    loop power
    movsd xmm0,xmm2
done:
    ret 

In [ ]:
%%file asmLimite.c
extern double asmLimite(int N);

In [ ]:
! nasm -f elf64 -g asmLimite.asm -o asmLimite.o
! gcc -shared asmLimite.o asmLimite.c -o asmLimite.so

Definimos la función con ctypes en python

In [ ]:
def ctypes_asmLimite():
    # ruta de la shared library
    lib = ctypes.CDLL('./asmLimite.so')
    
    # tipo de los argumentos
    lib.asmLimite.argtypes = [
        ctypes.c_int
    ]

    # definimos el tipo de salida de la función
    lib.asmLimite.restype = ctypes.c_double

    # se devuelve la función configurada
    return lib.asmLimite

In [ ]:
asmLimite = ctypes_asmLimite()

Probamos nuestra función

In [ ]:
print("Valor:",end = " ")
print(value)
var = asmLimite(value)
print("Limite:",end = " ")
print(var)

# Comparación de tiempos de ejecución

In [ ]:
def filtro_mediana(sign, ventana):
    signc = []
    tam = len(sign)
    offs = int((ventana-1)/2)
    for i in range(tam):
        inicio = i - offs if (i - offs > 0) else i
        fin = i + offs if (i + offs < tam) else tam
        signc.append(statistics.median(sign[inicio:fin]))
    return signc

In [ ]:
# size of arrays
sizes = 2**np.arange(11,18)

# tiempo en C
time_c = np.zeros((len(sizes),1),dtype = np.float64)

# tiempo con ruido en C
time_c_noise = np.zeros((len(sizes),1),dtype = np.float64)

# tiempo en ASM
time_asm = np.zeros((len(sizes),1),dtype = np.float64)

# tiempo con ruido en ASM
time_asm_noise = np.zeros((len(sizes),1),dtype = np.float64)

In [ ]:
veces = 50
tam = sizes[0]
index = range(veces)
tcp = []
tasmp = []

for i in range(veces):

    # entradas
    xp = np.random.randint(50)
    
    # tiempo C
    t = time.time()
    cLimite(xp)
    tcp.append(time.time() - t)
    
    # tiempo ASM
    t = time.time()
    asmLimite(xp)
    tasmp.append(time.time() - t)

In [ ]:
plt.plot(index, tcp, 'r-', label='tiempo C')
plt.plot(index, tasmp, 'b-', label='tiempo ASM')
plt.xlabel('tamagno')
plt.ylabel('tiempo')
plt.legend()
plt.show()

# ¿Qué sucede si N tiende a infinito?

In [ ]:
veces = 1000
tam = sizes[0]
index = range(veces)
c_inf = []
asm_inf = []
step = 100
for i in index:
    c_inf.append(cLimite(i))
    
    #tasmp.append(asmLimite(i))

In [ ]:
plt.plot(index, c_inf, 'r-', label='Limite C')
#plt.plot(index, tasmp, 'b-', label='Limite ASM')
plt.ylabel('f(n)')
plt.xlabel('n')
plt.legend()
plt.show()

In [ ]:
var = cLimite(100000)
print("Limite:",end = " ")
print(var)

# SpeedUp

In [ ]:
c_mean = statistics.mean(tcp)
asm_mean = statistics.mean(tasmp)

In [ ]:
speed_up = c_mean/asm_mean

In [ ]:
speed_up